## Solving Ordinary Differential Equations Numerically

**Nick Kern**
<br>
**Astro 9: Python Programming in Astronomy**
<br>
**UC Berkeley**

Reading: Chp. 8, Computational Physics w/ Python, Newman

In this lecture, we are going to be talking about how to apply the concepts we've been building with numerical integration and differentiation to solving *ordinary differential equations*. A differential equation is just an equation that relates two variables together via a derivative. For example, one of the simplest DE is the equation of motion of an object with a constant acceleration:

\begin{align}
\frac{dx}{dt} = at + v_{0}
\end{align}

which can easily be solved by-hand using separation of variables and integrating to get $x(t)$.

The general form of a first-order, ordinary differential equation looks like

\begin{align}
\frac{dx}{dt} = f(x, t),
\end{align}

in which case, the independent variable is $t$ and the dependent variable is $x$. You may have noticed if you did the first integral in your head that in order to actually solve these equations, we need to not only integrate it but also provide a boundary condition or an initial condition (the integration constant!).

Why do we care to solve an ODE numerically? In the case of our first example, we don't, really, because we can do that simply by-hand. However, there are *plenty* of equations in Astrophysics that either cannot be solved analytically, are just too tedious to be solved by-hand, or are neither but need to just be evaluated many, many times, in which case, we'd like a computer to do it for us!

One good example of an applicable (second-order) differential equation is one that describes the motion of an object subject to a Newtonian gravitational force:

\begin{align}
\vec{F} = ma = m\frac{d^{2}x}{dt^{2}} = -\frac{GMm}{r^{2}}{\hat{r}}
\end{align}

This form of equation applies not only to something like a planet orbiting a star, but also, for example, to a sattelite orbiting the Earth or to molecular dynamics. Solutions to equations of this form are generally applicable outside of just gravitational physics, and are particularly useful for determining the equation of motion of charged objects subject to an electric field because the electric force follows the same functional form.

Let's first start by discussing ways we can solve first-order ODEs.

### Euler's Method

Suppose we are given an equation of the form $\frac{dx}{dt} = f(x, t)$ and are given an initial condition of $x$ given some time $t_{1}$. **Our goal is to solve for the function $x(t)$** for all times $t$.

Using a Taylor expansion, we can write the value of $x$ at some later time as:

\begin{align}
x(t + \Delta t) &= x(t) + \Delta t\frac{dx}{dt} + \tfrac{1}{2}{\Delta t}^{2}\frac{d^{2}x}{dt^{2}} + \ldots \\
\\
&= x(t) + \Delta t\cdot f(x, t) + \mathcal{O}(\Delta t^{2})
\end{align}

If we take $\Delta t$ to be small, than the first two terms of this equation may be a good approximation to the true answer:

\begin{align}
x(t + \Delta t) \simeq x(t) + \Delta t\cdot f(x, t)
\end{align}

Note that what we have above is nothing more than just an extrapolation of a function, $x(t)$, based on its first derivative, which we call $f(x, t) = \frac{dx}{dt}$.

In order to solve for $x$ at some later time $t_{2}$, we need only divide up our timeline from $t_{1} < t < t_{2}$ into a bunch of small chunks and use the above method *at each chunk* to reach our answer of $x(t_{2})$. This is called *Euler's Method*.

**Example**

Let's use Euler's method to solve the differential equation

\begin{align}
\frac{dx}{dt} = -x^{3} + \sin(t)
\end{align}

with an initial condition of $x = 0$ when $t=0$. Find the value of $x(t = 10)$.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# define first derivative function
def f(x, t):
    return -x**3 + np.sin(t)
    

# Define bounds and step-sizes
t1 = 0
t2 = 10
N = 100
dt = float(t2-t1)/N

# Make t-points
tpoints = np.linspace(t1, t2, N+1)

# Define initial condition
x = 0

# Iterate Euler's Method to get x(t)
xpoints = []
for t in tpoints:
    # append value of x to xpoints
    xpoints.append(x)
    
    # update value using Euler's method
    x += dt * f(x, t)

In [ ]:
# figure
fig = plt.figure(figsize=(7,7))

# axes
ax = fig.add_subplot(1, 1, 1)
ax.grid(True)
ax.set_xlim(-1,11)
ax.set_ylim(-1.3,1.3)
ax.set_xlabel('t', fontsize=18)
ax.set_ylabel('x(t)', fontsize=18)

# plot
ax.plot(tpoints, xpoints)

You can see from our Taylor expansion argument that the error term in Euler's method scales as $\mathcal{O}(\Delta t^{2})$. This is the error induced *at each point* in our iteration. You can see that each point relies on the answer from the previous point, meaning error in compounded all the way to our final answer, $x(t_{2}$). To get the error on our final answer, we need to sum the error term of each point:

\begin{align}
\epsilon_{t_{2}} &= \sum_{k=0}^{N-1}\tfrac{1}{2}{\Delta t}^{2}\left(\frac{d^{2}x}{dt^{2}}\right)_{t=t_{k}} = \tfrac{1}{2}\Delta t\sum\Delta t\left(\frac{df}{dt}\right)\\
\\
&\simeq \tfrac{1}{2}\Delta t\int_{t_{1}}^{t_{2}}\frac{df}{dt}dt = \boxed{\tfrac{1}{2}\Delta t\left[f(x_{t_{2}},t_{2}) - f(x_{t_{1}}, t_{1})\right]}
\end{align}

Note that in this case, the error on the final answer $x(t_{2})$ only scales as $\Delta t^{1}$, which is worse than the error we accumulate per-point from Euler's method.

### Breakout 1

1.
Put your above script into a function called `ode_solve` that takes as input the derivative function $f$, the initial condition of $x_1$ and $t_1$, the stopping time $t_2$ and the number of time steps $N$ between $t_1$ and $t_2$. Have it return the two arrays `tpoints` and `xpoints`.

2.
Use your function to solve for $x(t)$ up to $t_2 = 10$ for initial conditions of

* $x = 0,\ t_1 = 0$
* $x = 1,\ t_1 = 0$
* $x = -1,\ t_1 = 0$
* $x = 0,\ t_1 = 2$

and plot all of these curves on a single plot.

In [ ]:
# define first derivative function
def f(x, t):
    return -x**3 + np.sin(t)

# define ode solver using Euler's Method
def ode_solve(f, x1, t1, t2, N=100):
    
    
    
    
    return tpoints, xpoints

### Second-Order Runge-Kutta

Euler's method may work well for our simple toy-model problem over a modest range of times. However, for more complex problems the poor error properties of Euler's method will prove fatal. Let's see if we can devise a more accurate algorithm for solving an ODE.

Runge-Kutta methods are a class of methods for solving ODE's. The first-order RK method is, for example, just Euler's method. Higher order methods, like second-order and fourth-order methods, are more accurate than first-order RK and are therefore of great importance.

Let's think back to our discussion of numerical derivatives. We saw two basic methods for performing a numerical derivative: the forward difference and central difference. The forward difference is similar to what we are doing in Euler's method, however, we saw that the central difference was much more accurate in estimating the derivative. The idea behind second-order Runge-Kutta is to use a central difference for the derivative, rather than the forward difference.

<img src='imgs/rk2.png' width=400px/>
<center>Figure 8.2 of Newman</center>

Mathematically, this means we invoke a Taylor expansion not around $t$, but around $t + \Delta t/2$, meaning we can get the value at $x(t+\Delta t)$ as

\begin{align}
x(t+\Delta t) = x(t + \tfrac{1}{2}\Delta t) + \tfrac{1}{2}\Delta t\left(\frac{dx}{dt}\right)_{t + \tfrac{1}{2}\Delta t} + \tfrac{1}{8}\Delta t^{2}\left(\frac{d^{2}x}{dt^{2}}\right)_{t + \tfrac{1}{2}\Delta t} + \mathcal{O}(\Delta t^{3}).
\end{align}

Similarly, we get can the value of $x(t)$ using our expansion centered at $t + \Delta t/2$ as

\begin{align}
x(t) = x(t + \tfrac{1}{2}\Delta t) - \tfrac{1}{2}\Delta t\left(\frac{dx}{dt}\right)_{t + \tfrac{1}{2}\Delta t} + \tfrac{1}{8}\Delta t^{2}\left(\frac{d^{2}x}{dt^{2}}\right)_{t + \tfrac{1}{2}\Delta t} + \mathcal{O}(\Delta t^{3}).
\end{align}

If we take the difference of these two equations, we are left with

\begin{align}
x(t+\Delta t) = x(t) + \Delta t\cdot f(x,t+\tfrac{1}{2}\Delta t) + \mathcal{O}(\Delta t^{3})
\end{align}

Notice two major things: (1) the first two terms is similar to our Euler method from before, except now the error term goes not as $\Delta t^{2}$ but as $\Delta t^{3}$ (which is why this is called "second-order" RK) and (2) the evaluation of our derivative function $f$ is no longer at $t$ but at $t + \frac{1}{2}\Delta t$, which we don't know! We can get around this by using our Euler's method from before to make the approximation $x(t + \frac{1}{2}\Delta t) = x(t) + \frac{1}{2}\Delta t\cdot f(x,t)$, and then insert that result into $f$ to complete the second-order Runge-Kutta method.

The full method can therefore be written as

\begin{align}
k_1 &= \Delta t\cdot f(x, t)\\
k_2 &= \Delta t\cdot f(x+\tfrac{1}{2}k_1, t + \tfrac{1}{2}\Delta t)\\
x(t + \Delta t) &= x(t) + k_2
\end{align}



**Example**

Let's modify our `ode_solve` function from before to incorporate the second-order Runge-Kutta method (aka, RK2). It is almost identical, with the exception of a few extra lines.

In [ ]:
# define first derivative function
def f(x, t):
    return -x**3 + np.sin(t)

# Define bounds and step-sizes
t1 = 0.0
t2 = 10.0
N = 10
dt = float(t2-t1)/N

# Make t-points
tpoints = np.linspace(t1, t2, N+1)

# Define initial condition
x = 0.0

# Iterate Euler's Method to get x(t)
xpoints = []
for t in tpoints:
    # append value of x to xpoints
    xpoints.append(x)
    
    # update value using RK2 method
    


In [ ]:
# plot
fig = plt.figure(figsize=(7,7))

ax = fig.add_subplot(1, 1, 1)



### Fourth-Order Runge-Kutta

We can continue to expand on our previous argument of taking central differences rather than forward differences to approximate the derivatives. The more sophisticated we get, however, the more complicated the equations become. A commonly agreed upon "sweet-spot" is the *fourth-order Runge-Kutta* method. We won't go through the derivation here, which is quite complicated and follows the same logic as RK2, but will quote the result for us to use. This fourth-order Runge-Kutta method (aka. RK4) has an error term that scales as $\mathcal{O}(\Delta t^{5})$, making it two orders better than RK2. The equations look like:

\begin{align}
k_1 &= \Delta t\cdot f(x, t)\\
k_2 &= \Delta t\cdot f(x + \tfrac{1}{2}k_1, t + \tfrac{1}{2}\Delta t)\\
k_3 &= \Delta t\cdot f(x + \tfrac{1}{2}k_2, t + \tfrac{1}{2}\Delta t)\\
k_4 &= \Delta t\cdot f(x + k_3, t + \Delta t)\\
\\
x(t + \Delta t) &= \boxed{x(t) + \tfrac{1}{6}(k_1 + 2k_2 + 3k_3 + k_4)}
\end{align}

### Breakout 2

1.
Take the above script for the second-order Runge-Kutta method and modify it to use the fourth-order RK method. Put it into a function called `RK4`, similar to your `ode_solve` function, which takes the same parameters.

2.
Use your `RK4` function to solve the same differential equation from before: $f(x, t) = \frac{dx}{dt} = -x^{3} + \sin(t)$ and make a plot of it. How many steps $N$ do we need to converge to a stable answer, as compared to what we got with Euler's method?

In [ ]:
def RK4(f, x1, t1, t2, N=10):
    
    
    
    
    return tpoints, xpoints

### Differential Equations with more than one Dependent Variable

Up to now we have studied differential equations with one independent variable and one dependent variable. Often, however, problems we encounter have more than one dependent variable. We call these *simultaneous differential equations*. In this case, we have multiple dependent variables, but still only have a single independent variable. Take for example the following set of equations

\begin{align}
\frac{dx}{dt} = xy - x;\ \ \frac{dy}{dt} = y - xy + \sin^{2}(\omega t),
\end{align}

where $x$ and $y$ are the two dependent variables and $t$ is our single independent variable. We can generalize a set of simultaneous DE to the form

\begin{align}
\frac{dx}{dt} = f_{x}(x,y,t);\ \ \frac{dy}{dt} = f_{y}(x,y,t)
\end{align}

which can further be condensed using vector notation, $\boldsymbol{r} = \langle x, y\rangle;\ \boldsymbol{f} = \langle f_{x}, f_{y}\rangle$, yielding

\begin{align}
\frac{d\boldsymbol{r}}{dt} = \boldsymbol{f}(\boldsymbol{r}, t).
\end{align}

Using this notation, simultaneous differential equations take the same form as before. Euler's method for simultaneous DE, for example, can be written as

\begin{align}
\boldsymbol{r}(t + \Delta t) = \boldsymbol{r}(t) + \Delta t\cdot\boldsymbol{f}(\boldsymbol{r},t),
\end{align}

where $\boldsymbol{r}$ could hold as many dependent variables as we like: $\boldsymbol{r} = \langle x, y, z, \ldots\rangle$.

Similarly, our fourth-order Runge-Kutta equations can be written in vector form as

\begin{align}
\boldsymbol{k}_1 &= \Delta t\cdot\boldsymbol{f}(\boldsymbol{r}, t)\\
\boldsymbol{k}_2 &= \Delta t\cdot\boldsymbol{f}(\boldsymbol{r}+\tfrac{1}{2}\boldsymbol{k}_1, t+\tfrac{1}{2}\Delta t)\\
\boldsymbol{k}_3 &= \Delta t\cdot\boldsymbol{f}(\boldsymbol{r}+\tfrac{1}{2}\boldsymbol{k}_1, t+\tfrac{1}{2}\Delta t)\\
\boldsymbol{k}_4 &= \Delta t\cdot\boldsymbol{f}(\boldsymbol{r}+\tfrac{1}{2}\boldsymbol{k}_1, t+\tfrac{1}{2}\Delta t)\\
\\
\boldsymbol{r}(t + \Delta t) &= \boldsymbol{r}(t) + \tfrac{1}{6}\left(\boldsymbol{k}_1 + 2\boldsymbol{k}_2 + 2\boldsymbol{k}_3 + \boldsymbol{k}_4\right)
\end{align}

**Example **

Let's apply our RK4 method in vector form to our simultaneous DE listed above.

In [ ]:
## Write Function
omega = 1.0
def f(r, t):
    # break into components
    x = r[0]
    y = r[1]
    # get derivatives
    fx = x*y - x
    fy = y - x*y + np.sin(omega * t)**2
    # return vector of derivatives
    return np.array([fx, fy], dtype=np.float)

# set bounds
t1 = 0.0
t2 = 10.0
N = 100
dt = float(t2 - t1) / N

# Initialize points in time
tpoints = np.linspace(t1, t2, N+1)

# Initialize empty x and y lists
xpoints = []
ypoints = []

# set initial condition
r = np.array([1.0, 1.0])

# loop over points in time
for t in tpoints:
    # append to x & y lists
    xpoints.append( r[0] )
    ypoints.append( r[1] )
    
    # calculate RK4
    k1 = dt * f(r, t)
    k2 = dt * f(r + 0.5*k1, t + 0.5*dt)
    k3 = dt * f(r + 0.5*k2, t + 0.5*dt)
    k4 = dt * f(r + k3, t + dt)
    r += (k1 + 2*k2 + 2*k3 + k4) / 6.0

In [ ]:
# plot
fig = plt.figure(figsize=(9,9))

# ax1
ax1 = fig.add_subplot(2, 1, 1)
ax1.grid(True)
ax1.set_ylabel('x(t)', fontsize=18)
ax1.plot(tpoints, xpoints, color='steelblue', linewidth=3)
ax.set_ylim(0.8,2.5)

# ax2
ax2 = fig.add_subplot(2, 1, 2)
ax2.grid(True)
ax2.set_xlabel('t', fontsize=20)
ax2.set_ylabel('y(t)', fontsize=18)
ax2.plot(tpoints, ypoints, color='darkorange', linewidth=3)
ax2.set_ylim(0.3,1.8)

We can also make a 3D plot using the techniques we learned in the maplotlib lecture!

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# 3d plot
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(1, 1, 1, projection='3d')

# axes
ax.set_xlabel('t',fontsize=15)
ax.set_ylabel('x',fontsize=15)
ax.set_zlabel('y',fontsize=15)
ax.set_xlim(0,10)
ax.set_ylim(0.8, 2.5)
ax.set_zlim(0.3, 1.8)

# plot
ax.scatter(tpoints, xpoints, ypoints, c='steelblue')

### Second-Order Differential Equations

Up to now we have studied differential equations that depend only on the first derivative with respect to the independent variable. However, we already know that some of the most important equations in physics--namely the relation between an object's motion and an acceleration induced by a force: $a(t) = \frac{d^{2}x}{dt^{2}}$--involve second derivatives. Now that we have studied how to solve first-order DE, we can begin to look at how to solve second-order differential equations.

The general form for a 2nd order DE, similar to our 1st order DE, can be written as

\begin{align}
\frac{d^{2}x}{dt^{2}} = f(x, \tfrac{dx}{dt}, t),
\end{align}

which is to say that our 2nd order term can depend on the dependent and independent variables $x$ & $t$, as well as the 1st order differential $\frac{dx}{dt}$. This could look like, for example, something along the lines of

\begin{align}
\frac{d^{2}x}{dt^{2}} = \frac{1}{x}\left(\frac{dx}{dt}\right)^{2} + 2\frac{dx}{dt} - x^{3}e^{-4t}
\end{align}

The trick to solving this equation is to define a new variable for our first derivative term, and then define our second order derivative as the first derivative of that variable. In other words:

\begin{align}
\frac{dx}{dt} &= y\\
\frac{d^{2}x}{dt^{2}} &= \frac{dy}{dt} = f(x,y,t)
\end{align}

This has reduced a single second-order DE into two simultaneous first-order DEs, which we know how to solve!

### Breakout 3:

The equation of motion for a point mass at the end of a massless and frictionless arm attached to a pivot can be written as

\begin{align}
\frac{d^{2}\theta}{dt^{2}} = -\frac{g}{\ell}\sin\theta
\end{align}
where $\theta$ is the angle displacement from center, $g$ is the gravitational acceleration and $\ell$ is the length of the arm. In the limit that $\theta << 1$ this can be linearized and solved exactly. In general, however, this is a nonlinear equation that cannot be solved exactly. We can, however, solve this on a computer by using the techniques we learned above.

Start by creating a new variable `omega` such that

\begin{align}
\frac{d\theta}{dt} &= \omega\\
&\text{and}\\
\frac{d\omega}{dt} &= -\frac{g}{\ell}\sin\theta.
\end{align}

Then, use the technique we studied above for a simultaneous set of first-order differential equations to solve for the trajectory of $\theta(t)$.

You can assume $\ell = 0.1$ meters. I have some code to get you started.

In [ ]:
g = 9.81
l = 0.1

def f(r, t):
    theta = r[0]
    omega = r[1]
    ftheta = omega
    fomega = -(g/l)*np.sin(theta)
    return np.array([ftheta, fomega], dtype=float)
